In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
# Read the CSV file
df = pd.read_csv('FT_History.csv')

df.head()

In [ ]:
class Transaction:
    def __init__(self, symbol, quantity, price, action, date):
        self.symbol = symbol
        self.quantity = quantity
        self.price = price
        self.action = action
        self.date = datetime.strptime(date, '%Y-%m-%d')


class Portfolio:
    def __init__(self):
        self.holdings = {}
        self.history = []
        self.cash = 0.0

    def process_transaction(self, transaction):
        symbol = transaction.symbol
        quantity = transaction.quantity
        price = transaction.price
        action = transaction.action

        if action == 'BUY':
            if symbol not in self.holdings:
                self.holdings[symbol] = {'quantity': 0, 'total_cost': 0}
            self.holdings[symbol]['quantity'] += quantity
            self.holdings[symbol]['total_cost'] += quantity * price
            self.cash -= quantity * price

        elif action == 'SELL' and symbol in self.holdings:
            self.holdings[symbol]['quantity'] -= quantity
            self.holdings[symbol]['total_cost'] -= quantity * price
            self.cash += quantity * price
            if self.holdings[symbol]['quantity'] <= 0:
                del self.holdings[symbol]

    def process_financial(self, amount):
        self.cash += amount

    def record_daily_balance(self, date):
        date = datetime.strptime(
            date, '%Y-%m-%d') if isinstance(date, str) else date
        daily_balance = next(
            (entry for entry in self.history if entry['date'] == date), None)
        if daily_balance:
            daily_balance['holdings'] = {k: v.copy()
                                         for k, v in self.holdings.items()}
            daily_balance['cash'] = self.cash
        else:
            self.history.append({
                'date': date,
                'holdings': {k: v.copy() for k, v in self.holdings.items()},
                'cash': self.cash
            })

    def get_history(self):
        return self.history

In [ ]:
# Process the transactions
portfolio = Portfolio()
for _, row in df.iterrows():
    if pd.notna(row['Symbol']) and row['Action'] in ['BUY', 'SELL']:
        transaction = Transaction(
            symbol=row['Symbol'].strip(),
            quantity=abs(float(row['Quantity'])),
            price=float(row['Price']) if pd.notna(row['Price']) else 0.0,
            action=row['Action'],
            date=row['TradeDate']
        )
        portfolio.process_transaction(transaction)
    elif row['RecordType'] == 'Financial':
        amount = float(row['Amount'])
        portfolio.process_financial(amount)

    portfolio.record_daily_balance(row['TradeDate'])
    
# Get the history of the portfolio
history = portfolio.get_history()

In [ ]:
# oldest_portfolio = history[:3]

# oldest_portfolio

In [ ]:
latest_portfolio = history[-1]

latest_portfolio

In [ ]:
def calculate_gains(symbol, quantity, total_cost, date=None):
    stock = yf.Ticker(symbol)
    date = date or datetime.today().strftime('%Y-%m-%d')
    date = datetime.strptime(date, '%Y-%m-%d')

    start_date = (date - timedelta(days=5)).strftime('%Y-%m-%d')
    end_date = (date + timedelta(days=1)).strftime('%Y-%m-%d')

    history = stock.history(start=start_date, end=end_date)['Close']
    if len(history) < 2:
        return {key: None for key in ['Current Price', 'Current Value', 'Day\'s Gain', 'Day\'s Gain %', 'Total Gain', 'Total Gain %']}

    latest_price = history.iloc[-1]
    previous_price = history.iloc[-2]

    current_value = quantity * latest_price
    previous_value = quantity * previous_price

    days_gain = current_value - previous_value
    days_gain_percent = (days_gain / previous_value) * \
        100 if previous_value != 0 else 0

    total_gain = current_value - total_cost
    total_gain_percent = (total_gain / total_cost) * \
        100 if total_cost != 0 else 0

    return {
        'Current Price': latest_price,
        'Current Value': current_value,
        'Day\'s Gain': days_gain,
        'Day\'s Gain %': days_gain_percent,
        'Total Gain': total_gain,
        'Total Gain %': total_gain_percent
    }


def get_portfolio_performance(date=None, print_results=True):
    if date is None:
        portfolio_on_date = latest_portfolio
        date_str = datetime.today().strftime('%Y-%m-%d')
    else:
        parsed_date = datetime.strptime(date, '%Y-%m-%d')
        # Find the closest date in history
        closest_date = min(history, key=lambda x: abs(
            x['date'] - parsed_date))['date']
        portfolio_on_date = next(
            (entry for entry in history if entry['date'] == closest_date), None)
        date_str = closest_date.strftime('%Y-%m-%d')

    results = {}
    for symbol, data in portfolio_on_date['holdings'].items():
        results[symbol] = calculate_gains(
            symbol, data['quantity'], data['total_cost'], date_str)

    df_results = pd.DataFrame(results).T
    df_results.index.name = 'Symbol'
    df_results = df_results.reset_index()

    df_results['Quantity'] = [portfolio_on_date['holdings']
                              [symbol]['quantity'] for symbol in df_results['Symbol']]
    df_results['Total Cost'] = [portfolio_on_date['holdings']
                                [symbol]['total_cost'] for symbol in df_results['Symbol']]

    column_order = ['Symbol', 'Quantity', 'Current Price', 'Current Value',
                    'Total Cost', 'Day\'s Gain', 'Day\'s Gain %', 'Total Gain', 'Total Gain %']
    df_results = df_results[column_order]

    df_results = df_results.sort_values(by='Symbol')

    total_portfolio_value = df_results['Current Value'].sum()
    total_days_gain = df_results['Day\'s Gain'].sum()
    total_days_gain_percent = (
        total_days_gain / total_portfolio_value) * 100 if total_portfolio_value != 0 else 0

    if print_results:
        print(df_results.to_string(index=False,
              float_format=lambda x: f'{x:.2f}'))
        print()
        print(f"Total Balance: ${total_portfolio_value:.2f}")
        print(f"Day's gain: ${total_days_gain:.2f} ({
              total_days_gain_percent:.2f}%)")
    else:
        return df_results, total_portfolio_value, total_days_gain, total_days_gain_percent


# Ensure all dates in history are converted to datetime objects
for entry in history:
    if isinstance(entry['date'], str):
        entry['date'] = datetime.strptime(entry['date'], '%Y-%m-%d')

In [ ]:
get_portfolio_performance()

In [ ]:
get_portfolio_performance('2023-09-10')

In [ ]:
def get_closest_date(target_date, available_dates):
    return min(available_dates, key=lambda x: abs(x - target_date))


def fetch_symbol_history(symbols, start_date, end_date):
    data = {}
    for symbol in symbols:
        ticker = yf.Ticker(symbol)
        data[symbol] = ticker.history(start=start_date, end=end_date)[
            'Close'].tz_localize(None)
        if datetime.now().time() < datetime.strptime('16:00', '%H:%M').time():
            current_price = ticker.history(period='1d')['Close'].iloc[-1]
            data[symbol].loc[datetime.now()] = current_price
    return data


def calculate_daily_performance(history, symbol_data, exclude_etfs=None):
    exclude_etfs = exclude_etfs or ['QQQ', 'SPY', 'NVDL']
    daily_performance = []
    last_portfolio = history[-1]
    end_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

    for date in pd.date_range(start=history[0]['date'], end=end_date):
        date = date.to_pydatetime()
        current_portfolio = next((entry for entry in reversed(
            history) if entry['date'] <= date), last_portfolio)

        total_value = total_cost = 0
        for symbol, data in current_portfolio['holdings'].items():
            if symbol in exclude_etfs:
                continue
            available_dates = symbol_data[symbol].index
            closing_price = symbol_data[symbol].loc[date if date in available_dates else get_closest_date(
                date, available_dates)]

            total_value += data['quantity'] * closing_price
            total_cost += data['total_cost']

        daily_return = total_value - total_cost
        daily_return_percent = (daily_return / total_cost) * \
            100 if total_cost != 0 else 0
        daily_performance.append({
            'date': date,
            'total_value': total_value,
            'daily_return': daily_return,
            'daily_return_percent': daily_return_percent
        })

    return daily_performance


def plot_daily_performance(daily_performance, sp500_data):
    dates = [entry['date'] for entry in daily_performance]
    total_values = [entry['total_value'] for entry in daily_performance]
    daily_return_percents = [entry['daily_return_percent']
                             for entry in daily_performance]

    fig, ax1 = plt.subplots(figsize=(16, 9))

    ax1.set_xlabel('Date')
    ax1.set_ylabel('Total Value', color='tab:blue')
    ax1.plot(dates, total_values, color='tab:blue', label='Total Value')
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    ax2 = ax1.twinx()
    ax2.set_ylabel('Daily Return %', color='tab:green')
    ax2.plot(dates, daily_return_percents,
             color='tab:green', label='Daily Return %', linewidth=3)
    ax2.tick_params(axis='y', labelcolor='tab:green')
    ax2.axhline(0, color='gray', linestyle='--', linewidth=1)

    min_return = min(daily_return_percents)
    max_return = max(daily_return_percents)
    current_return = daily_return_percents[-1]

    for return_value, color, label in [(min_return, 'red', 'Min'), (max_return, 'green', 'Max'), (current_return, 'blue', 'Current')]:
        date = dates[daily_return_percents.index(return_value)]
        ax2.plot(date, return_value, 'o', color=color)
        ax2.text(date, return_value, f'{label}: {return_value:.2f}%\n{date.strftime("%Y-%m-%d")}',
                 fontsize=10, color=color, ha='left')

    ax3 = ax1.twinx()
    ax3.spines['right'].set_position(('outward', 60))
    ax3.set_ylabel('S&P 500', color='tab:orange')
    ax3.plot(sp500_data.index, sp500_data.values,
             color='tab:orange', label='S&P 500')
    ax3.tick_params(axis='y', labelcolor='tab:orange')

    fig.tight_layout()
    plt.title('Daily Portfolio Performance vs S&P 500')
    plt.show()


# Ensure all dates in history are converted to datetime objects
for entry in history:
    if isinstance(entry['date'], str):
        entry['date'] = datetime.strptime(entry['date'], '%Y-%m-%d')

# Main execution
symbols = {symbol for entry in history for symbol in entry['holdings'].keys()}

start_date = history[0]['date'].strftime('%Y-%m-%d')
end_date = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
symbol_data = fetch_symbol_history(symbols, start_date, end_date)

# Fetch S&P 500 data
sp500_data = yf.Ticker('^GSPC').history(
    start=start_date, end=end_date)['Close'].tz_localize(None)
if datetime.now().time() < datetime.strptime('16:00', '%H:%M').time():
    sp500_current_price = yf.Ticker('^GSPC').history(period='1d')[
        'Close'].iloc[-1]
    sp500_data.loc[datetime.now()] = sp500_current_price

daily_performance = calculate_daily_performance(
    history, symbol_data, exclude_etfs=['QQQ', 'SPY', 'NVDL'])
    
plot_daily_performance(daily_performance, sp500_data)

In [ ]:
# Filter daily performance for year-to-date
current_year = datetime.now().year
ytd_performance = [
    entry for entry in daily_performance if entry['date'].year == current_year]

# Filter S&P 500 data for year-to-date
ytd_sp500_data = sp500_data[sp500_data.index.year == current_year]

# Plot year-to-date performance
plot_daily_performance(ytd_performance, ytd_sp500_data)

In [ ]:
daily_performance[-3:]